# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 27 2023 1:19PM,258263,19,9213291,"Emersa Waterbox, LLC",Released
1,Feb 27 2023 1:16PM,258262,19,60021495,"GCH Granules USA, Inc.",Released
2,Feb 27 2023 1:16PM,258262,19,60021514,"GCH Granules USA, Inc.",Released
3,Feb 27 2023 1:16PM,258262,19,60021509,"GCH Granules USA, Inc.",Released
4,Feb 27 2023 1:16PM,258262,19,60021500,"GCH Granules USA, Inc.",Released
5,Feb 27 2023 1:16PM,258262,19,60021492,"GCH Granules USA, Inc.",Released
6,Feb 27 2023 1:16PM,258261,10,Yusen-9213202,Yusen – 3D Matrix,Released
7,Feb 27 2023 1:16PM,258261,10,Yusen-9213218,Yusen – 3D Matrix,Released
8,Feb 27 2023 1:16PM,258261,10,Yusen-9213221,Yusen – 3D Matrix,Released
9,Feb 27 2023 1:16PM,258261,10,Yusen-9213252,Yusen – 3D Matrix,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,258255,Released,9
42,258256,Released,1
43,258261,Released,11
44,258262,Released,5
45,258263,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258255,NaN,NaN,9.0
258256,NaN,NaN,1.0
258261,NaN,NaN,11.0
258262,NaN,NaN,5.0
258263,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258149,0.0,0.0,18.0
258153,0.0,30.0,1.0
258162,0.0,0.0,58.0
258166,4.0,13.0,0.0
258188,0.0,0.0,24.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258149,0,0,18
258153,0,30,1
258162,0,0,58
258166,4,13,0
258188,0,0,24


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258149,0,0,18
1,258153,0,30,1
2,258162,0,0,58
3,258166,4,13,0
4,258188,0,0,24


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258149,,,18
1,258153,,30,1
2,258162,,,58
3,258166,4,13,
4,258188,,,24


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC"
1,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc."
6,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix
17,Feb 27 2023 1:05PM,258255,19,"GCH Granules USA, Inc."
22,Feb 27 2023 1:05PM,258255,19,Eywa Pharma Inc.
26,Feb 27 2023 1:05PM,258256,22,"NBTY Global, Inc."
27,Feb 27 2023 1:05PM,258254,22,"NBTY Global, Inc."
28,Feb 27 2023 12:54PM,258250,19,"GCH Granules USA, Inc."
33,Feb 27 2023 12:53PM,258249,19,"GCH Granules USA, Inc."
34,Feb 27 2023 12:37PM,258248,10,"Methapharm, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC",,,1
1,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc.",,,5
2,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix,,,11
3,Feb 27 2023 1:05PM,258255,19,"GCH Granules USA, Inc.",,,9
4,Feb 27 2023 1:05PM,258255,19,Eywa Pharma Inc.,,,9
5,Feb 27 2023 1:05PM,258256,22,"NBTY Global, Inc.",,,1
6,Feb 27 2023 1:05PM,258254,22,"NBTY Global, Inc.",,,1
7,Feb 27 2023 12:54PM,258250,19,"GCH Granules USA, Inc.",,1,4
8,Feb 27 2023 12:53PM,258249,19,"GCH Granules USA, Inc.",,,1
9,Feb 27 2023 12:37PM,258248,10,"Methapharm, Inc.",,,15


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC",1,,
1,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc.",5,,
2,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix,11,,
3,Feb 27 2023 1:05PM,258255,19,"GCH Granules USA, Inc.",9,,
4,Feb 27 2023 1:05PM,258255,19,Eywa Pharma Inc.,9,,
5,Feb 27 2023 1:05PM,258256,22,"NBTY Global, Inc.",1,,
6,Feb 27 2023 1:05PM,258254,22,"NBTY Global, Inc.",1,,
7,Feb 27 2023 12:54PM,258250,19,"GCH Granules USA, Inc.",4,1,
8,Feb 27 2023 12:53PM,258249,19,"GCH Granules USA, Inc.",1,,
9,Feb 27 2023 12:37PM,258248,10,"Methapharm, Inc.",15,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC",1,,
1,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc.",5,,
2,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix,11,,
3,Feb 27 2023 1:05PM,258255,19,"GCH Granules USA, Inc.",9,,
4,Feb 27 2023 1:05PM,258255,19,Eywa Pharma Inc.,9,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
1,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc.",5.0,NaN,NaN
2,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix,11.0,NaN,NaN
3,Feb 27 2023 1:05PM,258255,19,"GCH Granules USA, Inc.",9.0,NaN,NaN
4,Feb 27 2023 1:05PM,258255,19,Eywa Pharma Inc.,9.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 27 2023 1:19PM,258263,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
1,Feb 27 2023 1:16PM,258262,19,"GCH Granules USA, Inc.",5.0,0.0,0.0
2,Feb 27 2023 1:16PM,258261,10,Yusen – 3D Matrix,11.0,0.0,0.0
3,Feb 27 2023 1:05PM,258255,19,"GCH Granules USA, Inc.",9.0,0.0,0.0
4,Feb 27 2023 1:05PM,258255,19,Eywa Pharma Inc.,9.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4389483,335.0,36.0,0.0
12,258245,0.0,1.0,0.0
15,774611,46.0,78.0,20.0
16,258201,1.0,0.0,0.0
19,3357033,48.0,27.0,4.0
20,258196,0.0,1.0,0.0
21,258234,1.0,0.0,0.0
22,516510,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4389483,335.0,36.0,0.0
1,12,258245,0.0,1.0,0.0
2,15,774611,46.0,78.0,20.0
3,16,258201,1.0,0.0,0.0
4,19,3357033,48.0,27.0,4.0
5,20,258196,0.0,1.0,0.0
6,21,258234,1.0,0.0,0.0
7,22,516510,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,335.0,36.0,0.0
1,12,0.0,1.0,0.0
2,15,46.0,78.0,20.0
3,16,1.0,0.0,0.0
4,19,48.0,27.0,4.0
5,20,0.0,1.0,0.0
6,21,1.0,0.0,0.0
7,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,335.0
1,12,Released,0.0
2,15,Released,46.0
3,16,Released,1.0
4,19,Released,48.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,20.0,0.0,4.0,0.0,0.0,0.0
Executing,36.0,1.0,78.0,0.0,27.0,1.0,0.0,0.0
Released,335.0,0.0,46.0,1.0,48.0,0.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,20.0,0.0,4.0,0.0,0.0,0.0
1,Executing,36.0,1.0,78.0,0.0,27.0,1.0,0.0,0.0
2,Released,335.0,0.0,46.0,1.0,48.0,0.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,0.0,0.0,20.0,0.0,4.0,0.0,0.0,0.0
1,Executing,36.0,1.0,78.0,0.0,27.0,1.0,0.0,0.0
2,Released,335.0,0.0,46.0,1.0,48.0,0.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()